In [ ]:
import random

from model import GeneratorModule
from scripts.datasetscripts import parse_concatenated_json, format_augmented_goal
import torch
import ast

In [ ]:
ckpt_path = "/Users/artem.semidetnov/Desktop/MetaGenerator_v1/.cadence/cache/id2f7519f525f7457f899938c06c46d14c/45769/outputs/lightning_logs/version_6/checkpoints/last.ckpt"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GeneratorModule.load_from_checkpoint(ckpt_path, strict=False)
model.to(device)
model.eval()

In [ ]:
test_path = '/Users/artem.semidetnov/Desktop/MetaGenerator_v1/datasets/dataset_small_with_addons/test.json'

test_prompts = parse_concatenated_json(test_path)
def test_model(prompt_index : int):
    prompt = test_prompts[prompt_index]

    prompt_ = format_augmented_goal(
            prompt["Expected type"],
            prompt["Premises"],
            512,
            0.0,
        )
    prompts = [prompt_]
    enc = model.tokenizer(
        prompts,
        padding=True,
        truncation=True,
        max_length=getattr(model.hparams, "max_inp_seq_len", 512),
        return_tensors="pt",
    )
    enc = {k: v.to(device) for k, v in enc.items()}
    with torch.no_grad():
        outputs = model.generator.generate(
            input_ids=enc["input_ids"],
            attention_mask=enc["attention_mask"],
            num_beams=4,#getattr(model.hparams, "num_beams", 4),
            num_return_sequences=4,
            length_penalty=getattr(model.hparams, "length_penalty", 0.0),
            max_length=getattr(model.hparams, "max_oup_seq_len", 128),
        )

    preds = model.tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return preds, prompt['Expression']

In [ ]:
# len_test = len(parse_concatenated_json(test_path)) #759

In [ ]:
with open('testoutputs', 'w') as f:
    for i in range(700):
        f.write(str(test_model(i)) + "\n")

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Failed to parse line 1: an integer is required
Failed to parse line 2: an integer is required
Failed to parse line 3: an integer is required
Failed to parse line 4: an integer is required
Failed to parse line 5: an integer is required
2 6


In [35]:
# !pip3 install tree-sitter-python
cnt_total = 0
cnt_correct = 0

import ast
import os
from codebleu import bleu, weighted_ngram_match
import codebleu

# Robust CodeBLEU calculation to handle tree-sitter failures
def calc_codebleu_robust(references, predictions, lang="python", weights=(0.25, 0.25, 0.25, 0.25)):
    alpha, beta, gamma, theta = weights

    # Preprocess
    references = [[x.strip() for x in ref] if isinstance(ref, list) else [ref.strip()] for ref in references]
    hypothesis = [x.strip() for x in predictions]

    # Tokenizer
    def tokenizer(s):
        return s.split()

    tokenized_hyps = [tokenizer(x) for x in hypothesis]
    tokenized_refs = [[tokenizer(x) for x in reference] for reference in references]

    # 1. Ngram Match (BLEU)
    ngram_match_score = bleu.corpus_bleu(tokenized_refs, tokenized_hyps)

    # 2. Weighted Ngram Match
    weighted_ngram_match_score = 0.0
    try:
        package_dir = os.path.dirname(codebleu.__file__)
        keywords_path = os.path.join(package_dir, "keywords", f"{lang}.txt")
        if os.path.exists(keywords_path):
            with open(keywords_path, "r", encoding="utf-8") as f:
                keywords = [x.strip() for x in f.readlines()]
            
            def make_weights(reference_tokens, key_word_list):
                return {token: 1 if token in key_word_list else 0.2 for token in reference_tokens}

            tokenized_refs_with_weights = [
                [[reference_tokens, make_weights(reference_tokens, keywords)] for reference_tokens in reference]
                for reference in tokenized_refs
            ]
            weighted_ngram_match_score = weighted_ngram_match.corpus_bleu(tokenized_refs_with_weights, tokenized_hyps)
        else:
             weighted_ngram_match_score = ngram_match_score
    except Exception:
        weighted_ngram_match_score = ngram_match_score

    # 3. Syntax & Dataflow (Try-Except)
    syntax_match_score = 0.0
    dataflow_match_score = 0.0
    
    failed_tree_sitter = False
    if gamma > 0 or theta > 0:
        try:
            from codebleu import syntax_match, dataflow_match, utils
            tree_sitter_language = utils.get_tree_sitter_language(lang)
            
            if gamma > 0:
                syntax_match_score = syntax_match.corpus_syntax_match(
                    references, hypothesis, lang, tree_sitter_language=tree_sitter_language
                )
            if theta > 0:
                dataflow_match_score = dataflow_match.corpus_dataflow_match(
                    references, hypothesis, lang, tree_sitter_language=tree_sitter_language
                )
        except Exception:
            # print(f"Tree-sitter failed")
            failed_tree_sitter = True

    if failed_tree_sitter:
        # Renormalize weights: distribute gamma+theta to alpha+beta
        total_valid = alpha + beta
        if total_valid > 0:
            scale = (alpha + beta + gamma + theta) / total_valid
            alpha *= scale
            beta *= scale
        gamma = 0
        theta = 0

    score = (
        alpha * ngram_match_score
        + beta * weighted_ngram_match_score
        + gamma * syntax_match_score
        + theta * (dataflow_match_score or 1.0)
    )
    
    return score

# Safe CodeBLEU scorer with progressive fallbacks
def score_codebleu_safe(pred: str, correct: str, lang: str = "python"):
    if not isinstance(pred, str) or not isinstance(correct, str):
        return None, "failed"
    pred_s = pred.replace("\x00", "").strip()
    corr_s = correct.replace("\x00", "").strip()
    if not pred_s or not corr_s:
        return None, "empty"

    try:
        score = calc_codebleu_robust([corr_s], [pred_s], lang=lang, weights=(0.25, 0.25, 0.25, 0.25))
        score = max(0.0, min(1.0, score))
        return score, "robust_custom"
    except Exception as e:
        return None, "failed"

# For each line in testoutputs: compute the minimal CodeBLEU distance (1 - CodeBLEU score)
min_distances = []
max_scores = []  # 1 - min_distance (for convenience)
components_used = []  # Which components succeeded for the best score

with open("testoutputs", "r") as f:
    for idx, line in enumerate(f, start=1):
        cnt_total += 1

        # Parse a tuple: (preds: list[str], correct: str)
        try:
            parsed = ast.literal_eval(line)
        except Exception as e:
            print(f"Parse error on line {idx}: {e.__class__.__name__}: {e}")
            min_distances.append(None)
            max_scores.append(None)
            continue

        if not (isinstance(parsed, tuple) and len(parsed) == 2):
            print(f"Parse error on line {idx}: expected tuple (preds, correct), got {type(parsed)}")
            min_distances.append(None)
            max_scores.append(None)
            continue

        preds, correct = parsed
        if not isinstance(preds, list) or not all(isinstance(p, str) for p in preds) or not isinstance(correct, str):
            print(f"Parse error on line {idx}: wrong types. preds type={type(preds)}, correct type={type(correct)}")
            min_distances.append(None)
            max_scores.append(None)
            continue

        if correct in preds:
            cnt_correct += 1

        # Compute CodeBLEU score for each prediction; convert to distance = 1 - score
        best_score = None
        best_comp = ""
        for pred in preds:
            # Skip empty strings to avoid tokenizer issues
            if not pred.strip() or not correct.strip():
                continue
            score, label = score_codebleu_safe(pred, correct, lang="python")
            if score is None:
                # If CodeBLEU still fails on this pair, skip it and try others
                # Note: We keep the concise line-level message volume low.
                # Uncomment for detailed diagnostics:
                # print(f"Scoring error on line {idx}: fallback failed for a pair")
                continue
            if best_score is None or score > best_score:
                best_score = score
                best_comp = label

        if best_score is None:
            # No valid score could be computed for this line
            min_distances.append(None)
            max_scores.append(None)
            components_used.append("")
        else:
            max_scores.append(best_score)
            min_distances.append(1.0 - best_score)
            components_used.append(best_comp)

# Save minimal distances (and corresponding max scores) to a results file for later analysis
out_path = "testoutputs_min_codebleu.csv"
with open(out_path, "w") as out:
    out.write("line_index,min_codebleu_distance,max_codebleu_score,components_used\n")
    for i, (d, s, c) in enumerate(zip(min_distances, max_scores, components_used), start=1):
        out.write(f"{i},{'' if d is None else d},{'' if s is None else s},{c}\n")

print(f"Exact-match count: {cnt_correct} out of {cnt_total}")
computed = sum(1 for d in min_distances if d is not None)
print(f"Computed CodeBLEU for {computed} / {cnt_total} lines. Results saved to {out_path}")



Exact-match count: 234 out of 700
Computed CodeBLEU for 0 / 700 lines. Results saved to testoutputs_min_codebleu.csv


In [47]:
cnt_total = 0
cnt_correct = 0
from codebleu import calc_codebleu

import ast  # ensure this cell is self-contained even if earlier imports weren't run

# For each line in testoutputs: compute the minimal CodeBLEU distance (1 - CodeBLEU score)
min_distances = []
max_scores = []  # 1 - min_distance (for convenience)

with open("testoutputs", "r") as f:
    for idx, line in enumerate(f, start=1):
        cnt_total += 1
        if cnt_total > 50:
            break
        parsed = ast.literal_eval(line)

        preds, correct = parsed

        # print(list(map(lambda x : x.split(" ")[0], preds)))
        # print(correct.split(" ")[0])

        if correct.split(" ")[0] in list(map(lambda x : x.split(" ")[0], preds)) and not(correct in preds):
            print(correct.split(" ")[0], " | ", correct)
            cnt_correct += 1
cnt_correct

replicate  |  replicate s (zro )
<_*-left  |  <_*-left y<z z>0 x>0
<=<_<=  |  <=<_<= x'<=<V' idp
\lam  |  \lam _ => {?}
p  |  p Uy
inP  |  inP (1, idp, idp)
toZero  |  toZero {R} {g (l !! j).2} {conj a (f (l !! j).2)} p
tupleBeta  |  tupleBeta {P1}
\lam  |  \lam d => {?}
rewrite  |  rewrite (prod-id-right ) {?}
\lam  |  \lam p0 {p1} {p2} x*y=x*z => {?}
rewrite  |  rewrite (hinv_f {associator-iso }) {?}
\lam  |  \lam p0 p1 => {?}
limUnique  |  limUnique {?}
cauchy-subset  |  cauchy-subset p {?}
<=-uniform  |  <=-uniform u {?} d
func-cont  |  func-cont  OBall-open
<=<-left  |  <=<-left {?} p
\lam  |  \lam c => {?}


19